In [ ]:
# !pip install transformers==4.30

In [ ]:
# importing all libraries and loading Google Drive
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
import spacy
# Load English language model
nlp = spacy.load("en_core_web_sm")
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


In [ ]:
# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Load clean text data from the txt file
file_path = "/content/drive/MyDrive/newtask/cleaned_apt_data.txt"
with open(file_path, "r") as file:
    text_data = file.read()

In [ ]:
text_data

'\nYour request has been blocked. This could be\n                        due to several reasons.Skip\n                                                                to main   \n                                                                                                                                                \n                                                                                                                                                                        \n                                                                                                                                                                        \n                                                                                                                                                                        EdgePCs\n                                                                                                                                                        &\n    

In [ ]:
# Splitting text data into sentences
sentences = nltk.sent_tokenize(text_data)

# Keywords to filter sentences
keywords = ['APT29', 'CVE-2023-42793', 'Midnight Blizzard', 'TeamCity saga']

In [ ]:
sentences_refined = []

for sentence in sentences:
    # Primary check for keywords
    if any(keyword in sentence for keyword in keywords):
        # Ensure sentence is greater than 10 characters and does not contain more than 5 consecutive spaces
        if len(sentence) > 10 and not '     ' in sentence:
            if len(sentence) <= 200:  # Check if sentence is within desired length range
                sentences_refined.append(sentence)
            else:
                # Split long sentences into smaller chunks of max 200 characters
                chunks = [sentence[i:i+200] for i in range(0, len(sentence), 200)]
                for chunk in chunks:
                    # Ensure chunk length is within desired range and does not contain more than 5 consecutive spaces
                    if len(chunk) > 10 and not '     ' in chunk:
                        sentences_refined.append(chunk)

In [ ]:
len(sentences_refined)

134

In [ ]:
(sentences_refined[11])

'is a malware  employed by group APT29.'

In [ ]:
# Function to generate meaningful sentences
def generate_text(prompt, max_length=100, temperature=0.3):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)  # Set attention mask to 1 for all tokens
    pad_token_id = tokenizer.eos_token_id  # Set pad token ID to eos token ID
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, temperature=temperature, pad_token_id=pad_token_id, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [ ]:
# Example prompt
prompt = sentences_refined[11]

# Generate text based on the prompt
generated_sentence = generate_text(prompt)

print(generated_sentence)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


is a malware  employed by group APT29.

The malware is a Trojan that can be used to infect computers and other devices. It can be used to infect other computers and other devices.

The malware is a Trojan that can be used to infect computers and other devices. It can be used to infect other computers and other devices.

The malware is a Trojan that can be used to infect computers and other devices. It can be used to infect other computers and other devices


In [ ]:
# Initialize an empty list to store generated sentences
generated_sentences = []

# Loop through each prompt in sentences_refined
for prompt in sentences_refined:
    # Generate text based on the prompt
    generated_sentence = generate_text(prompt)

    # Check if the generated sentence contains any of the keywords
    if any(keyword in generated_sentence for keyword in keywords):
        # Append the generated sentence to the list
        generated_sentences.append(generated_sentence)

In [ ]:
# path to the newtask folder in your Google Drive
output_file_path = '/content/drive/My Drive/newtask/regenerated_sentences.txt'

# Save the generated sentences to a text file
with open(output_file_path, "w") as file:
    for sentence in generated_sentences:
        file.write(sentence + "\n")